COPYRIGHT © 2018 Kiran Arun <kironni@gmail.com>

### Setup

In [2]:
!rm -r Reinforcement_Learning-101-challenge
!git clone https://github.com/KiranArun/Reinforcement_Learning-101-challenge.git 1>/dev/null
!bash Reinforcement_Learning-101-challenge/scripts/setup.sh

rm: cannot remove 'Reinforcement_Learning-101-challenge': No such file or directory
Cloning into 'Reinforcement_Learning-101-challenge'...
remote: Counting objects: 75, done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 75 (delta 39), reused 45 (delta 17), pack-reused 0
Unpacking objects: 100% (75/75), done.
Extracting templates from packages: 100%


In [84]:
# DEFINITELY DO NOT EDIT THIS CELL

import os
# Start noVNC remote Xwindows in a browser with TurboVNC
!bash /content/Reinforcement_Learning-101-challenge/scripts/kill-novnc.sh >/dev/null
!sleep 2
# start novnc server and expose via ngrok NOT HTTPS
os.environ['PATH'] += "${PATH}:/opt/VirtualGL/bin:/opt/TurboVNC/bin"
# hack line below to get vnc to start as ipython subprocess
!timeout 5 /opt/websockify/run 5901 --web=/opt/noVNC --wrap-mode=ignore -- /opt/TurboVNC/bin/vncserver :1 -securitytypes otp -otp -noxstartup 2>/dev/null
get_ipython().system_raw('python /opt/websockify/run 5901 --web=/opt/noVNC --wrap-mode=ignore -- /opt/TurboVNC/bin/vncserver :1 -securitytypes otp -otp -noxstartup > /content/.vnc/stdout 2>&1 &')
# ngrok remote tunnel
get_ipython().system_raw('/content/ngrok http 5901 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print('XWindow Link:', json.load(sys.stdin)['tunnels'][0]['public_url'])"
# get Xwindows going (right click to get openbox menu)
get_ipython().system_raw('DISPLAY=:1 openbox &')
# one-time-password
!grep "one-time password:" /content/.vnc/stdout

XWindow Link: http://3f1d7a27.ngrok.io
Full control one-time password: 15267371


In [0]:
# DO NOT EDIT THIS CELL

# set up Xvfb and attach it's own vncserver and test with spinning gears
!pkill Xvfb   # clean-up
!pkill x11vnc # clean-up

get_ipython().system_raw('DISPLAY=:1 xterm &')
get_ipython().system_raw('/usr/bin/Xvfb :99 -screen 0 640x480x24 &')
get_ipython().system_raw('/usr/bin/x11vnc -rfbport 5902 -forever -display :99 &')
!sleep 2
get_ipython().system_raw('DISPLAY=:1 /usr/bin/vncviewer localhost:5902 &')
get_ipython().system_raw('DISPLAY=:99 glxgears &')

In [0]:
# open a terminal into the glxgears session using TightVNC
get_ipython().system_raw('DISPLAY=:99 xterm -e /bin/bash -l -c "pkill glxgears"')
get_ipython().system_raw('DISPLAY=:1 /usr/bin/vncviewer localhost:5902 &')
get_ipython().system_raw('DISPLAY=:99 xterm -geometry 105x35 &')

# RL Challenge

In [0]:
# importing libraries
import numpy as np
import matplotlib.pyplot as plt
import gym

import keras
import keras.layers as layers
from rl import agents,memory,policy

### Hyperparameters

In [0]:
# Hyperparams

# number of steps to keep in experience buffer
memory_limit = 150

# discount value
gamma = 0.99

# how much to update target graph
target_model_update = 5e-2

# learning rate
learning_rate = 5e-2

# number of steps to sample from buffer to train on
batch_size = 100

In [36]:
# DO NOT EDIT

# setting environment
env = gym.make('CartPole-v1')
num_actions = env.action_space.n

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


### Neural Network

In [37]:
keras.backend.clear_session()

# create neural net
nn = keras.models.Sequential()

# keep this as input layer
nn.add(layers.Flatten(input_shape=(1,) + env.observation_space.shape))

# keep this as output layer
nn.add(layers.Dense(num_actions))

nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


### Optimizer

https://keras.io/optimizers/

In [0]:
# set optimizer
optimizer = keras.optimizers.SGD(lr=learning_rate)

### Memory

In [0]:
# DO NOT EDIT

# set experience buffer
memory = memory.SequentialMemory(limit=memory_limit,
                                    window_length=1)

### Policy

choose from: https://github.com/keras-rl/keras-rl/blob/master/rl/policy.py

[info on choosing](https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-7-action-selection-strategies-for-exploration-d3a97b7cceaf)

In [0]:
# set policy
policy = policy.BoltzmannQPolicy()

### Agent

choose from: https://github.com/keras-rl/keras-rl/tree/master/rl/agents

and use: https://github.com/keras-rl/keras-rl/tree/master/examples

In [0]:
# create agent
model = agents.dqn.DQNAgent(model=nn,
                            nb_actions=num_actions,
                            memory=memory,
                            gamma=gamma,
                            batch_size=batch_size,
                            nb_steps_warmup=100,
                            target_model_update=target_model_update,
                            policy=policy)

In [0]:
# DO NOT EDIT

# compile model
model.compile(optimizer,
            metrics=['mae'])

In [43]:
# DO NOT EDIT

# train
history = model.fit(env,
                  nb_steps=1000,
                  verbose=2)

Training for 1000 steps ...
  17/1000: episode: 1, duration: 0.051s, episode steps: 17, steps per second: 335, episode reward: 17.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.353 [0.000, 1.000], mean observation: 0.082 [-1.020, 1.818], loss: --, mean_absolute_error: --, mean_q: --
  33/1000: episode: 2, duration: 0.010s, episode steps: 16, steps per second: 1645, episode reward: 16.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.312 [0.000, 1.000], mean observation: 0.097 [-1.220, 2.101], loss: --, mean_absolute_error: --, mean_q: --
  52/1000: episode: 3, duration: 0.011s, episode steps: 19, steps per second: 1681, episode reward: 19.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.368 [0.000, 1.000], mean observation: 0.073 [-1.006, 1.841], loss: --, mean_absolute_error: --, mean_q: --
  63/1000: episode: 4, duration: 0.007s, episode steps: 11, steps per second: 1486, episode reward: 11.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.818 [0.000, 1.000], 

 831/1000: episode: 44, duration: 0.114s, episode steps: 20, steps per second: 175, episode reward: 20.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.250 [0.000, 1.000], mean observation: 0.050 [-1.920, 2.949], loss: 2.684561, mean_absolute_error: 5.639667, mean_q: 10.083839
 842/1000: episode: 45, duration: 0.056s, episode steps: 11, steps per second: 198, episode reward: 11.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.091 [0.000, 1.000], mean observation: 0.156 [-1.718, 2.874], loss: 3.352817, mean_absolute_error: 5.766298, mean_q: 10.294446
 871/1000: episode: 46, duration: 0.136s, episode steps: 29, steps per second: 213, episode reward: 29.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.517 [0.000, 1.000], mean observation: -0.086 [-1.504, 0.654], loss: 2.997024, mean_absolute_error: 5.808767, mean_q: 10.391899
 883/1000: episode: 47, duration: 0.072s, episode steps: 12, steps per second: 167, episode reward: 12.000, mean reward: 1.000 [1.000, 1.000], mean

In [64]:
# DO NOT EDIT

# test
hist = model.test(env, nb_episodes=50, visualize=False)
print('mean:', np.mean(hist.history['episode_reward']))

Testing for 5 episodes ...
Episode 1: reward: 53.000, steps: 53
Episode 2: reward: 84.000, steps: 84
Episode 3: reward: 29.000, steps: 29
Episode 4: reward: 115.000, steps: 115
Episode 5: reward: 39.000, steps: 39
mean: 64.0


In [0]:
nn.save('/content/models/nn.hdf5', overwrite=True)